In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
% matplotlib 

Using matplotlib backend: TkAgg


In [2]:
data = np.loadtxt('DL07_milimill_09_xi.csv')
IDs, X, Y, Z = data[:,0], data[:,1], data[:,2], data[:,3]
#h = .73
#X = (X/h)*10**6
#Y = (Y/h)*10**6
#Z = (Z/h)*10**6

In [3]:
len(IDs)

14193

In [4]:
def logdist (r1,r2):
    x1,y1,z1 = r1
    x2,y2,z2 = r2
    Lbox = 62.5
    Lx, Ly, Lz = abs(x2-x1), abs(y2-y1), abs(z2-z1)
    if Lx >= Lbox/2: Lx = Lbox - Lx
    if Ly >= Lbox/2: Ly = Lbox - Ly
    if Lz >= Lbox/2: Lz = Lbox - Lz     
    d = (1./2)*np.log10(Lx**2 + Ly**2 + Lz**2 + 1e-10) 
    return d

def ToClass(d, NBIN, XMIN,XMAX):
    label = int((d-XMIN)/(XMAX-XMIN)*NBIN)
    return label
    
def RR (r,n,dr):
    V_c = (4./3)*np.pi*((10**(r+dr))**3 - (10**r)**3)
    V_t = 62.5**3
    return (V_c/V_t)*n**2
    
def ToDist (label_array, NBIN, XMIN, XMAX):
    d = (label_array*(XMAX-XMIN)/NBIN) + XMIN
    return d

In [5]:
start_time = time.time()

NBIN = 30
Dmin = -2  # Min separation (log(d)) [d] = Mpc
Dmax = 1.  # Max separation (log(d))
Binsize = (Dmax - Dmin)/NBIN

lenght = len(IDs)
DD_array = np.zeros(NBIN)
for i in range(lenght):
    r1 = np.array([X[i], Y[i], Z[i]])
    for j in range(i+1,lenght):
        r2 = np.array([X[j], Y[j], Z[j]])
        label = ToClass(logdist(r1,r2), NBIN, Dmin, Dmax)
        if (label >= NBIN) | (label < 0): continue
        DD_array[label] += 1
        
bins = np.linspace(Dmin, Dmax, NBIN, endpoint = False)   
dist_array = bins + Binsize *.5
RR_array = np.array([RR(r,lenght, Binsize) for r in bins])

print("--- %3fs: loaded ---" % (time.time() - start_time))

--- 1037.830404s: loaded ---


In [17]:
1037.8/60

17.296666666666667

In [6]:
(2*DD_array/RR_array)-1.

array([  6.04674466e+04,   1.66090266e+04,   1.24130760e+04,
         9.07547379e+03,   5.75863716e+03,   4.54042792e+03,
         2.93858590e+03,   2.07268036e+03,   1.60308991e+03,
         1.09413452e+03,   8.31022568e+02,   5.71900216e+02,
         3.96688575e+02,   2.82786201e+02,   1.92296358e+02,
         1.30278524e+02,   8.61923567e+01,   5.48970008e+01,
         3.46557013e+01,   2.08910893e+01,   1.23361996e+01,
         7.40583909e+00,   4.67367086e+00,   3.10322493e+00,
         2.03628261e+00,   1.46248357e+00,   1.23989995e+00,
         8.31475415e-01,   5.68172648e-01,   3.40236990e-01])

In [7]:
logxi = np.log10((2*DD_array/RR_array)-1)

In [8]:
logxi

array([ 4.78152163,  4.22034418,  4.09387941,  3.95786931,  3.76031972,
        3.65709679,  3.46813839,  3.31653233,  3.20495788,  3.03907072,
        2.91961282,  2.75732026,  2.59844969,  2.45145821,  2.28397106,
        2.11487283,  1.93546876,  1.73954862,  1.53977469,  1.31996109,
        1.09118139,  0.86957427,  0.66965812,  0.49181326,  0.30883805,
        0.165091  ,  0.09338664, -0.08015059, -0.24551968, -0.46821847])

In [9]:
DD_array, RR_array

(array([  1.04000000e+02,   5.70000000e+01,   8.50000000e+01,
          1.24000000e+02,   1.57000000e+02,   2.47000000e+02,
          3.19000000e+02,   4.49000000e+02,   6.93000000e+02,
          9.44000000e+02,   1.43100000e+03,   1.96600000e+03,
          2.72300000e+03,   3.87700000e+03,   5.26900000e+03,
          7.14000000e+03,   9.46200000e+03,   1.21030000e+04,
          1.54040000e+04,   1.88700000e+04,   2.29370000e+04,
          2.88460000e+04,   3.88480000e+04,   5.60570000e+04,
          8.27650000e+04,   1.33930000e+05,   2.43071000e+05,
          3.96557000e+05,   6.77483000e+05,   1.15527700e+06]),
 array([  3.43981054e-03,   6.86332434e-03,   1.36941324e-02,
          2.73233863e-02,   5.45173231e-02,   1.08776360e-01,
          2.17037372e-01,   4.33046490e-01,   8.64041342e-01,
          1.72398913e+00,   3.43981054e+00,   6.86332434e+00,
          1.36941324e+01,   2.73233863e+01,   5.45173231e+01,
          1.08776360e+02,   2.17037372e+02,   4.33046490e+02,
      

In [25]:
plt.plot(dist_array,logxi,'k.', label = r"$M > 10^{9} M_{\odot}$")
#plt.ylim(-2,6)
plt.xlabel("$log(d/h^{-1}Mpc)$")
plt.ylabel(r"$log(\xi)$")
plt.legend(frameon=False)
plt.show()

In [11]:
np.savetxt("xi_data_09.txt", np.array([dist_array, logxi]).T)